In [226]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

In [227]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [228]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [229]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

In [230]:
# columns for data normalize
df_market_data.columns

Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')

### Prepare the Data

In [231]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d', 
                                                                    'price_change_percentage_14d', 'price_change_percentage_30d', 
                                                                    'price_change_percentage_60d', 'price_change_percentage_200d', 
                                                                    'price_change_percentage_1y']])

In [232]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(market_data_scaled, columns=['price_change_percentage_24h', 'price_change_percentage_7d', 
                                                                    'price_change_percentage_14d', 'price_change_percentage_30d', 
                                                                    'price_change_percentage_60d', 'price_change_percentage_200d', 
                                                                    'price_change_percentage_1y'])

# Copy the crypto names from the original data
df_market_data_scaled['coinid'] = df_market_data.index
# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index('coinid')
# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coinid,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [233]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [234]:
# Create an empty list to store the inertia values
inertia = []
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_mod = KMeans(n_clusters=i, random_state=0, n_init='auto')
    k_mod.fit(df_market_data_scaled)
    inertia.append(k_mod.inertia_)

In [235]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict = {"k": k, "inertia": inertia}

# then put it in a df
df_elbow = pd.DataFrame(elbow_dict)

# plot with hvplot, all inertia values, visually determine optimal k
elbow_scaled = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    xticks=k,
    width=800,
    height=400,
    rot=90
)
elbow_scaled

:Curve   [k]   (inertia)

In [236]:
# it's nice that i can do this visually, but i'm wondering how can it be done numerically
# this might be awful, but i think my determining factor would be the difference in 
# slopes on either side of a given value
# slope calc using (y2-y1)/(x2-x1), storing slopes as list to be appended
slope_df = []
for i in range(1, len(df_elbow)):
    slope = (df_elbow['inertia'].iloc[i] - df_elbow['inertia'].iloc[i-1]) / (df_elbow['k'].iloc[i] - df_elbow['k'].iloc[i-1])
    slope_df.append(slope)
slope_df = [float('nan')] + slope_df
# add slopes to df_elbow
df_elbow['slopes'] = slope_df

# now i want to id the value of k where the difference of slopes on either side of that k is the greatest
kick_off = -1
for i in range(1, len(df_elbow)-1):
    abs_diff = abs(df_elbow['slopes'].iloc[i-1] - df_elbow['slopes'].iloc[i+1])
    # iterate to find the largest value, kick off at -1
    if abs_diff > kick_off:
        kick_off = abs_diff
        best_k = i

print(f"the best k value is: {best_k}")

the best k value is: 4


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** visually, 4 is the best value

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [237]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0, n_init='auto')

In [238]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, n_init='auto', random_state=0)

In [239]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_predict = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(crypto_predict)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [240]:
# Create a copy of the DataFrame
df_scaled_pred = df_market_data_scaled.copy()

In [241]:
# Add a new column to the DataFrame with the predicted clusters
df_scaled_pred['cluster'] = crypto_predict

# Display sample data
df_scaled_pred.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster
coinid,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [242]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scaled_plot_clusters = df_scaled_pred.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols = "coinid",
    rot=90
)
scaled_plot_clusters

:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coinid)

---

### Optimize Clusters with Principal Component Analysis.

In [243]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [244]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca.fit_transform(df_market_data_scaled)
# View the first five rows of the DataFrame. 
pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [245]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

[0.3719856  0.34700813 0.17603793]
0.8950316570309842


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** ~89.5%

In [246]:
# DataFrame with the PCA data
df_scaled_pca = pd.DataFrame(pca_data, columns=['Part One', 'Part Two', 'Part Three'])
# Copy the crypto names from the original data
df_scaled_pca['coinid'] = df_market_data_scaled.index
# Set the coinid column as index
df_scaled_pca = df_scaled_pca.set_index('coinid')
# Display sample data
df_scaled_pca.head()

,Part One,Part Two,Part Three
coinid,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [247]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [248]:
# Create an empty list to store the inertia values
inertia = []
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_mod = KMeans(n_clusters=i, random_state=0, n_init='auto')
    k_mod.fit(df_scaled_pca)
    inertia.append(k_mod.inertia_)

In [249]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict = {"k": k, "inertia": inertia}

# then put it in a df
df_elbow = pd.DataFrame(elbow_dict)

# plot with hvplot, all inertia values, visually determine optimal k
elbow_pca = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    xticks=k,
    width=800,
    height=400,
    rot=90
)
elbow_pca

:Curve   [k]   (inertia)

In [250]:
# numerically determine w/ slopes
# slope calc using (y2-y1)/(x2-x1), storing slopes as list to be appended
slope_df = []
for i in range(1, len(df_elbow)):
    slope = (df_elbow['inertia'].iloc[i] - df_elbow['inertia'].iloc[i-1]) / (df_elbow['k'].iloc[i] - df_elbow['k'].iloc[i-1])
    slope_df.append(slope)
slope_df = [float('nan')] + slope_df
# add slopes to df_elbow
df_elbow['slopes'] = slope_df

# now i want to id the value of k where the difference of slopes on either side of that k is the greatest
kick_off = -1
for i in range(1, len(df_elbow)-1):
    abs_diff = abs(df_elbow['slopes'].iloc[i-1] - df_elbow['slopes'].iloc[i+1])
    # iterate to find the largest value, kick off at -1
    if abs_diff > kick_off:
        kick_off = abs_diff
        best_k = i

print(f"the best k value is: {best_k}")

the best k value is: 4


#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The best k value does not differ. The inertia values decreased a little from the original set. Still, the weird part is that the slope calculations I recreated largely remained the same between points, hence why the numerical version with code achieves the same result.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [251]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0, n_init='auto')

In [252]:
# Fit the K-Means model using the PCA data
model.fit(df_scaled_pca)

KMeans(n_clusters=4, n_init='auto', random_state=0)

In [253]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_predict_pca = model.predict(df_scaled_pca)
# Print the resulting array of cluster values.
print(crypto_predict_pca)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [254]:
# Create a copy of the DataFrame with the PCA data
df_scaled_pca_pred = df_scaled_pca.copy()
# Add a new column to the DataFrame with the predicted clusters
df_scaled_pca_pred['cluster'] = crypto_predict_pca
# Display sample data
df_scaled_pca_pred.head()

,Part One,Part Two,Part Three,cluster
coinid,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [255]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_plot_clusters = df_scaled_pca_pred.hvplot.scatter(
    x="Part One",
    y="Part Two",
    by="cluster",
    hover_cols = "coinid",
    rot=90
)
pca_plot_clusters

:NdOverlay   [cluster]
   :Scatter   [Part One]   (Part Two,coinid)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [256]:
# Composite plot to contrast the Elbow curves; had to rename each the plots
(elbow_scaled + elbow_pca).cols(2)


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [257]:
# Composite plot to contrast the clusters
(scaled_plot_clusters + pca_plot_clusters).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coinid)
   .NdOverlay.II :NdOverlay   [cluster]
      :Scatter   [Part One]   (Part Two,coinid)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The critical difference is the positioning of the outliers between the plotting sets. Cluster 3 in the scaled data appears closer to clusters 0 and 2, whereas it moves away from all clusters entirely in the PCA data.